निम्नलिखित नोटबुक्स को चलाने के लिए, यदि आपने अभी तक नहीं किया है, तो आपको एक मॉडल डिप्लॉय करना होगा जो `text-embedding-ada-002` को बेस मॉडल के रूप में उपयोग करता हो और डिप्लॉयमेंट नाम को .env फाइल के अंदर `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` के रूप में सेट करना होगा।


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

अगला, हम एम्बेडिंग इंडेक्स को एक पांडा डेटा फ्रेम में लोड करने जा रहे हैं। एम्बेडिंग इंडेक्स एक JSON फ़ाइल में संग्रहित है जिसका नाम `embedding_index_3m.json` है। एम्बेडिंग इंडेक्स में अक्टूबर 2023 के अंत तक के प्रत्येक YouTube ट्रांसक्रिप्ट के लिए एम्बेडिंग्स शामिल हैं।


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

अगला, हम `get_videos` नामक एक फ़ंक्शन बनाएंगे जो क्वेरी के लिए एम्बेडिंग इंडेक्स में खोज करेगा। यह फ़ंक्शन उन शीर्ष 5 वीडियो को लौटाएगा जो क्वेरी के सबसे समान हैं। फ़ंक्शन इस प्रकार काम करता है:

1. सबसे पहले, एम्बेडिंग इंडेक्स की एक कॉपी बनाई जाती है।
2. इसके बाद, OpenAI एम्बेडिंग API का उपयोग करके क्वेरी के लिए एम्बेडिंग की गणना की जाती है।
3. फिर एम्बेडिंग इंडेक्स में `similarity` नामक एक नया कॉलम बनाया जाता है। `similarity` कॉलम में क्वेरी एम्बेडिंग और प्रत्येक वीडियो सेगमेंट के एम्बेडिंग के बीच कोसाइन समानता होती है।
4. इसके बाद, `similarity` कॉलम के आधार पर एम्बेडिंग इंडेक्स को फ़िल्टर किया जाता है। एम्बेडिंग इंडेक्स को केवल उन वीडियो तक सीमित किया जाता है जिनकी कोसाइन समानता 0.75 या उससे अधिक होती है।
5. अंत में, एम्बेडिंग इंडेक्स को `similarity` कॉलम के अनुसार क्रमबद्ध किया जाता है और शीर्ष 5 वीडियो लौटाए जाते हैं।


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

यह फ़ंक्शन बहुत सरल है, यह केवल खोज क्वेरी के परिणामों को प्रिंट करता है।


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. सबसे पहले, एम्बेडिंग इंडेक्स को एक पांडा डेटा फ्रेम में लोड किया जाता है।
2. इसके बाद, उपयोगकर्ता से एक क्वेरी दर्ज करने के लिए कहा जाता है।
3. फिर `get_videos` फ़ंक्शन को क्वेरी के लिए एम्बेडिंग इंडेक्स में खोज करने के लिए कॉल किया जाता है।
4. अंत में, `display_results` फ़ंक्शन को उपयोगकर्ता को परिणाम दिखाने के लिए कॉल किया जाता है।
5. फिर उपयोगकर्ता से एक और क्वेरी दर्ज करने के लिए कहा जाता है। यह प्रक्रिया तब तक चलती रहती है जब तक उपयोगकर्ता `exit` दर्ज नहीं करता।

![](../../../../translated_images/hi/notebook-search.1e320b9c7fcbb0bc.png)

आपसे एक क्वेरी दर्ज करने के लिए कहा जाएगा। एक क्वेरी दर्ज करें और एंटर दबाएं। एप्लिकेशन क्वेरी से संबंधित वीडियो की एक सूची लौटाएगा। एप्लिकेशन उस वीडियो में उस स्थान का लिंक भी लौटाएगा जहाँ प्रश्न का उत्तर स्थित है।

यहाँ कुछ क्वेरी हैं जिन्हें आप आज़मा सकते हैं:

- Azure Machine Learning क्या है?
- कन्वोल्यूशनल न्यूरल नेटवर्क कैसे काम करते हैं?
- न्यूरल नेटवर्क क्या है?
- क्या मैं Azure Machine Learning के साथ Jupyter Notebooks का उपयोग कर सकता हूँ?
- ONNX क्या है?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या अशुद्धियाँ हो सकती हैं। मूल दस्तावेज़ अपनी मूल भाषा में ही प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सलाह दी जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
